In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import sys
print(sys.executable)
!pwd

/cluster/datastore/ibrahimh/conda_envs/gpu_39_env1/bin/python
/cluster/datastore/ibrahimh/projects/ELLIPSE/dev_phase1


In [3]:
import pandas as pd

# Load the dataset
data = pd.read_csv('/cluster/datastore/ibrahimh/projects/ELLIPSE/dataset/train.csv')

# Preview the data
print(data.head())

# Check column names
print(data.columns)

        text_id                                          full_text  cohesion  \
0  0016926B079C  I think that students would benefit from learn...       3.5   
1  0022683E9EA5  When a problem is a change you have to let it ...       2.5   
2  00299B378633  Dear, Principal\n\nIf u change the school poli...       3.0   
3  003885A45F42  The best time in life is when you become yours...       4.5   
4  0049B1DF5CCC  Small act of kindness can impact in other peop...       2.5   

   syntax  vocabulary  phraseology  grammar  conventions  
0     3.5         3.0          3.0      4.0          3.0  
1     2.5         3.0          2.0      2.0          2.5  
2     3.5         3.0          3.0      3.0          2.5  
3     4.5         4.5          4.5      4.0          5.0  
4     3.0         3.0          3.0      2.5          2.5  
Index(['text_id', 'full_text', 'cohesion', 'syntax', 'vocabulary',
       'phraseology', 'grammar', 'conventions'],
      dtype='object')


In [4]:
from sklearn.model_selection import train_test_split

# Combine the 6 important labels into one column
data['combined_labels'] = data[['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

# Count occurrences of each combined label
label_counts = data['combined_labels'].value_counts()

# Split data into two groups: those with enough samples for stratification and those without
data_stratify = data[data['combined_labels'].isin(label_counts[label_counts >= 4].index)]
data_no_stratify = data[~data['combined_labels'].isin(label_counts[label_counts >= 4].index)]

# Stratify the data with enough samples
stratified_sample, _ = train_test_split(data_stratify, test_size=0.8, stratify=data_stratify['combined_labels'], random_state=42)

# Randomly sample from the rest (no stratification for rare cases)
non_stratified_sample, _ = train_test_split(data_no_stratify, test_size=0.8, random_state=42)

# Combine the two samples
final_sample = pd.concat([stratified_sample, non_stratified_sample])

# Drop the combined_labels column if you don't need it
final_sample = final_sample.drop(columns=['combined_labels'])

print(final_sample.shape)

(781, 8)


In [5]:
type(final_sample)

pandas.core.frame.DataFrame

In [8]:
# Randomly sample 100 rows from the DataFrame
random_samples = final_sample.sample(n=500, random_state=42)  # Use random_state for reproducibility

# Display the sampled DataFrame
print(random_samples)

           text_id                                          full_text  \
3629  F4BE7BB77A43  Tomas Jefferson say is wonderful how much may ...   
217   1087B6C7FB98  I say its a good idea that the school menu sho...   
2362  AF96A2176FB7  Do you think students should have online class...   
1287  62DCD859B7FB  I undestand the company Author Ralph Waldo Eme...   
2889  D14413B502FD  We accomplish more when we are always doing so...   
...            ...                                                ...   
2218  A54391563C0F  The best way to spend your time outside is goi...   
1566  78018FB48420  Students should not be able to graduate early....   
2074  9BF717F45D68  Did you know that one of our students got 1000...   
2774  CB71E5F9AE1E  Do you think failure is a good thing or bad th...   
1430  6D7377CA8608  Technology allows people to work in an easy wa...   

      cohesion  syntax  vocabulary  phraseology  grammar  conventions  
3629       2.5     2.5         3.5          2.5    

In [9]:
# Save the stratified sample to a CSV file
random_samples.to_csv('/cluster/datastore/ibrahimh/projects/ELLIPSE/dataset/stratified_500.csv', index=False)

In [ ]:
#end

In [11]:
# Save the stratified sample to a CSV file
final_sample.to_csv('./dataset/stratified_sample.csv', index=False)

In [10]:
!pwd

/cluster/datastore/ibrahimh/projects/ELLIPSE
